## LLMChain
- memory에서 기억하고 있는 내용을 chain에 주입하는 방법 중 하나임.
    - Off-the-shell memory로서, 범용적인 chain이다.
    
- 여기서는, memory를 chatting을 위한 chain에 주입하려고 한다.

### 문자열 형태의 prompt를 이용

In [10]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import (
    PromptTemplate, 
    ChatPromptTemplate,
    MessagesPlaceholder
)

In [3]:
llm = ChatOpenAI(temperature=0.2)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80
)

In [9]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True
)

chain.predict(question="My name is jiho!")



> Entering new LLMChain chain...
Prompt after formatting:
My name is jiho!

> Finished chain.


'Hello Jiho! How can I assist you today?'

In [10]:
chain.predict(question="I live in Seoul!")



> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul!

> Finished chain.


"That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"

In [11]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


"I'm sorry, I do not have access to that information."

- 나는 분명하게 이름을 말했는데, LLM은 이전의 내용을 전혀 기억하지 못하고 있다!

In [12]:
memory.load_memory_variables({})

{'history': "System: Jiho introduces themselves as living in Seoul, and the AI responds positively, highlighting the vibrant and bustling city with a rich history and culture. The AI expresses hope that Jiho is enjoying their time living in Seoul. When Jiho reveals their name, the AI greets them warmly and asks how it can assist. The human shares their excitement about living in Seoul, mentioning the variety of things to see and do in the city.\nHuman: What is my name?\nAI: I'm sorry, I do not have access to that information."}

- memory에는 분명하게 이전 대화 기록이 남아있음을 확인할 수 있다.
- 따라서, 채팅 LLM이 memory의 내용을 인식할 수 있도록 추가적인 조치를 취해야 한다!

#### 해결책
- 1. 템플릿에 대화 기록을 저장할 공간(Placeholder)를 마련한다.
- 2. memory 객체 생성 시, memory_key 파라미터를 통해 placeholder 이름을 명시한다.


In [4]:
llm = ChatOpenAI(temperature=0.2)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,
    memory_key="chat_history"
)

template = """
You are a helpful AI talking to human!

{chat_history}
Human: {question}
You:        
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question="My name is jiho!")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to human!


Human: My name is jiho!
You:        


> Finished chain.


'Hello Jiho! How can I assist you today?'

In [5]:
chain.predict(question="I live in Seoul!")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to human!

Human: My name is jiho!
AI: Hello Jiho! How can I assist you today?
Human: I live in Seoul!
You:        


> Finished chain.


"That's great to hear! Seoul is a vibrant and bustling city. Is there anything specific you would like to know or talk about regarding Seoul?"

In [6]:
chain.predict(question="Gwangjang market is very famous traditional market in Seoul! There are lots of star foods including Spicy rice cake, beef tartare and so on.")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to human!

Human: My name is jiho!
AI: Hello Jiho! How can I assist you today?
Human: I live in Seoul!
AI: That's great to hear! Seoul is a vibrant and bustling city. Is there anything specific you would like to know or talk about regarding Seoul?
Human: Gwangjang market is very famous traditional market in Seoul! There are lots of star foods including Spicy rice cake, beef tartare and so on.
You:        


> Finished chain.


'Yes, Gwangjang Market is indeed a famous traditional market in Seoul known for its delicious street food offerings. Spicy rice cakes (tteokbokki) and beef tartare (yukhoe) are definitely popular choices among visitors. Have you had the chance to try any of these dishes at Gwangjang Market?'

In [7]:
chain.predict(question="Yeah, i went to there to eat yukhoe with my friend! Very delicious! Amazing!")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to human!

System: The human introduces themselves as Jiho and mentions they live in Seoul. The AI acknowledges this and asks if there is anything specific they would like to know or talk about regarding Seoul. Jiho mentions Gwangjang market as a famous traditional market in Seoul, highlighting the various star foods available such as Spicy rice cake and beef tartare.
AI: Yes, Gwangjang Market is indeed a famous traditional market in Seoul known for its delicious street food offerings. Spicy rice cakes (tteokbokki) and beef tartare (yukhoe) are definitely popular choices among visitors. Have you had the chance to try any of these dishes at Gwangjang Market?
Human: Yeah, i went to there to eat yukhoe with my friend! Very delicious! Amazing!
You:        


> Finished chain.


"That's great to hear, Jiho! Yukhoe is definitely a tasty dish to try at Gwangjang Market. It's wonderful that you had a delicious experience there with your friend. Is there any other street food or dish you would recommend trying at Gwangjang Market?"

### 메시지 형태 프롬프트를 사용

In [11]:
llm = ChatOpenAI(temperature=0.2)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,
    memory_key="chat_history",
    return_messages=True
)

# MessagesPlaceholder 객체를 통해, 이전 대화 기록을 저장한다.
prompt = ChatPromptTemplate.from_messages(
    messages=[
        ("system", "You are a helpful AI talking to human!"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}")
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

chain.predict(question="My name is jiho!")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to human!
Human: My name is jiho!

> Finished chain.


'Hello Jiho! How can I assist you today?'

In [12]:
chain.predict(question="I live in Seoul!")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to human!
Human: My name is jiho!
AI: Hello Jiho! How can I assist you today?
Human: I live in Seoul!

> Finished chain.


"Seoul is a vibrant city with a rich history and culture. Is there anything specific you'd like to know or talk about regarding Seoul?"

In [13]:
chain.predict(question="Gwangjang market is very famous traditional market in Seoul! There are lots of star foods including Spicy rice cake, beef tartare and so on.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to human!
Human: My name is jiho!
AI: Hello Jiho! How can I assist you today?
Human: I live in Seoul!
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you'd like to know or talk about regarding Seoul?
Human: Gwangjang market is very famous traditional market in Seoul! There are lots of star foods including Spicy rice cake, beef tartare and so on.

> Finished chain.


'Yes, Gwangjang Market is indeed a popular traditional market in Seoul known for its delicious street food and traditional Korean dishes. Spicy rice cake (tteokbokki) and beef tartare (yukhoe) are definitely must-try items when visiting the market. Have you had a chance to try them yourself?'

In [14]:
chain.predict(question="Yeah, i went to there to eat yukhoe with my friend! Very delicious! Amazing!")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to human!
System: The human introduces themselves as Jiho and mentions they live in Seoul. The AI acknowledges this and talks about Seoul's vibrant culture and history. The human then brings up Gwangjang market, a famous traditional market in Seoul, and mentions some of the popular foods there like Spicy rice cake and beef tartare.
AI: Yes, Gwangjang Market is indeed a popular traditional market in Seoul known for its delicious street food and traditional Korean dishes. Spicy rice cake (tteokbokki) and beef tartare (yukhoe) are definitely must-try items when visiting the market. Have you had a chance to try them yourself?
Human: Yeah, i went to there to eat yukhoe with my friend! Very delicious! Amazing!

> Finished chain.


"That's wonderful to hear, Jiho! Yukhoe is a delicacy in Korean cuisine and it's great that you got to experience it at Gwangjang Market with your friend. The market is a fantastic place to explore and enjoy a variety of authentic Korean dishes. If you have any other favorite foods or places in Seoul, feel free to share!"

In [15]:
memory.load_memory_variables({})

{'chat_history': [SystemMessage(content="The human introduces themselves as Jiho and mentions they live in Seoul. The AI acknowledges this and talks about Seoul's vibrant culture and history. The human then brings up Gwangjang market, a famous traditional market in Seoul, and mentions some of the popular foods there like Spicy rice cake and beef tartare. The AI agrees and highlights the delicious street food and traditional Korean dishes at the market. Jiho shares their experience of trying beef tartare (yukhoe) at Gwangjang Market with a friend and describes it as very delicious and amazing."),
  AIMessage(content="That's wonderful to hear, Jiho! Yukhoe is a delicacy in Korean cuisine and it's great that you got to experience it at Gwangjang Market with your friend. The market is a fantastic place to explore and enjoy a variety of authentic Korean dishes. If you have any other favorite foods or places in Seoul, feel free to share!")]}